In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
from loguru import logger
import numpy as np
import time
import pickle
import h5py
from itkwidgets import view

# My imports (se si modifica il file bisogna riavviare tutto il kernel)
from utilities import plot_all_slices_notzero,plot_slices,dice_coef

# Scapole

In [ ]:
with open("processing/scapula_clustering_labels.pkl","rb") as f:
    labels = pickle.load(f)

In [ ]:
left_scapulae_clusters = [2,3,4,7,8,10,12,13] # Non sicurissimo di destra e sinistra in realtà, potrebbero essere invertite, però non importa
right_scapulae_clusters = [0,1,5,6,9,11,14]
wrong = [8,   37,   63,   67,   68,  107,  137,  151,  165,  183,  206,
         207,  208,  209,  217,  222,  225,  227,  254,  265,  266,  278,
         280,  281,  293,  298,  305,  327,  336,  373,  386,  395,  396,
         450,  496,  501,  519,  542,  570,  580,  622,  623,  669,  685,
         695,  718,  719,  731,  737,  742,  743,  751,  758,  772,  777,
         787,  794,  795,  796,  814,  826,  841,  874,  876,  878,  879,
         882,  896,  911,  920,  922,  924,  928,  929,  960,  989, 1023,
        1031, 1048] # Scapole che erano nei cluster di sinistra ma sono di destra

In [ ]:
print(wrong)

In [ ]:
right_scapulae_indexes=[]
left_scapulae_indexes=[]
for i,lab in enumerate(labels):
    if i not in wrong:
        if lab in right_scapulae_clusters:
            right_scapulae_indexes.append(i)
        if lab in left_scapulae_clusters:
            left_scapulae_indexes.append(i)
    else: # Se è in wrong so che è destra ma era stata messa in cluster di sinistra
        right_scapulae_indexes.append(i)

In [ ]:
len(right_scapulae_indexes),len(left_scapulae_indexes) # Meglio flippare le sinistre quindi che sono meno

In [ ]:
print(left_scapulae_indexes)

In [ ]:
dataset_el_to_ct_scap.index(553) in left_scapulae_indexes

In [ ]:
with open("processing/scapula_flipped_indexes.pkl","wb") as f:
    pickle.dump(left_scapulae_indexes,f)

Flippo le scapole sinistre e le faccio diventare destre

In [ ]:
st=time.time()
with h5py.File("dataset_scapula_labelmap_res.hdf5","r") as f:
    scapula_dataset = f.get("mydataset")[:]
print(f"Tempo lettura dataset: {time.time()-st:.2f} sec")

In [ ]:
for index in left_scapulae_indexes:
    scapula_dataset[index] = np.flip(scapula_dataset[index],axis=0)

with h5py.File("dataset_scapula_labelmap_res_flipped.hdf5","w") as f:
    data_set = f.create_dataset("mydataset",data=scapula_dataset)

# Omeri

Ora devo trovare le TC e gli indici corrispondenti per gli omeri

In [ ]:
with open("scapula_preprocessing/arrays/pos_array_hor_all.pkl","rb") as f1:
    pos_array_hor_all_scap = pickle.load(f1)
dataset_el_to_ct_scap = [] # In corrispondenza dell'indice l'elemento conterrà la CT corrispondente
for i,el in enumerate(pos_array_hor_all_scap,start=1):
    if len(el) != 0:
        dataset_el_to_ct_scap.append(i)
    if len(el) > 2:
        dataset_el_to_ct_scap.append(i)

In [ ]:
with open("humerus_preprocessing/arrays/pos_array_hor_all.pkl","rb") as f1:
    pos_array_hor_all_hum = pickle.load(f1)
dataset_el_to_ct_hum = [] # In corrispondenza dell'indice l'elemento conterrà la CT corrispondente
for i,el in enumerate(pos_array_hor_all_hum,start=1):
    if len(el) != 0:
        dataset_el_to_ct_hum.append(i)
    if len(el) > 2:
        dataset_el_to_ct_hum.append(i)

In [ ]:
len(dataset_el_to_ct_scap),len(dataset_el_to_ct_hum)

In [ ]:
left_humerus_indexes = []
for index in left_scapulae_indexes:
    left_shoulder_ct = dataset_el_to_ct_scap[index]
    left_humerus_indexes.append(dataset_el_to_ct_hum.index(left_shoulder_ct))

In [ ]:
print(left_humerus_indexes)

In [ ]:
with open("processing/humerus_flipped_indexes.pkl","wb") as f:
    pickle.dump(left_humerus_indexes,f)

In [ ]:
st=time.time()
with h5py.File("dataset_humerus_labelmap_res.hdf5","r") as f:
    humerus_dataset = f.get("mydataset")[:]
print(f"Tempo lettura dataset: {time.time()-st:.2f} sec")

In [ ]:
humerus_dataset.shape

In [ ]:
for index in left_humerus_indexes:
    humerus_dataset[index] = np.flip(humerus_dataset[index],axis=0)

with h5py.File("dataset_humerus_labelmap_res_flipped.hdf5","w") as f:
    data_set = f.create_dataset("mydataset",data=humerus_dataset)